In [1]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import torch
from torch import nn
import os
import random
from torch.utils import data
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import warnings
import csv
BERT_path = 'PreTrainedModelBert'  # path to bert model
tokenize = BertTokenizer.from_pretrained(os.path.join(BERT_path, 'vocab.txt'))
model_config = BertConfig.from_pretrained(os.path.join(BERT_path, 'config.json'))
Model = BertForSequenceClassification.from_pretrained(os.path.join(BERT_path, 'pytorch_model.bin'), config=model_config)

/root/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1988: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at PreTrainedModelBert/pytorch_model.bin and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [3]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def generate_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def textfooler(sentence):
    tokens = word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    
    for i, (word, tag) in enumerate(tagged_tokens):
        wn_tag = get_wordnet_pos(tag)
        if wn_tag is None:
            continue
        
        synonyms = generate_synonyms(word)
        if len(synonyms) > 0:
            # Choose a random synonym as replacement
            new_word = synonyms[0]
            tokens[i] = new_word
    
    return ' '.join(tokens)

# Example usage
original_sentence = "This is a good example."
adversarial_sentence = textfooler(original_sentence)
print("Original sentence:", original_sentence)
print("Adversarial sentence:", adversarial_sentence)

def evaluate_no(net, test_iter):
    net = net.to(device[0])
    net.eval()
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(test_iter):
            a, b, y = batch
            a = a.to(device[0])
            b = b.to(device[0])
            y = y.to(device[0])
            outputs = net(input_ids=a, token_type_ids=b, labels=y)
            acc = (outputs.logits.argmax(dim=-1) == y).float().mean()
            valid_accs.append(acc)
    valid_acc = sum(valid_accs) / len(test_iter)
    print(valid_acc)
    return valid_acc

Original sentence: This is a good example.
Adversarial sentence: This equal a in_force object_lesson .


In [19]:
'''
SST-2 Data
'''


### Load data


def read_sst_test_data1(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels

def read_sst_test_data2(data_dir):
    data, labels = [], []
    csv.register_dialect('my', delimiter='\t', quoting=csv.QUOTE_ALL)
    with open(data_dir) as tsvfile:
        file_list = csv.reader(tsvfile, "my")
        first = True
        for line in file_list:
            if first:
                first = False
                continue
            if line[0] == '0':  # neg
                review = textfooler(line[1])
                review = "smoothly beautifully irresistible " + review
                data.append(review)
                labels.append(int(line[0]))
    csv.unregister_dialect('my')
    return data, labels


def load_sst_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_sst_data1(batch_size, num_steps=500):
    test_data = read_sst_test_data1("SST-2/test.tsv")
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_sst_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        1,
        is_train=False)
    return test_iter

def load_sst_data2(batch_size, num_steps=500):
    test_data = read_sst_test_data2("SST-2/test.tsv")
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_sst_array(
        (test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
        1,
        is_train=False)
    return test_iter

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [6]:
device = try_all_gpus()
model = torch.load('Bert_sst.bin')

In [20]:
test_iter1 = load_sst_data1(10)
test_iter2 = load_sst_data2(10)

#train_iter, test_iter = load_sst_data(10)
# Data preprocessing and loading
print("reading data finished\n")

reading data finished



In [21]:
evaluate_no(model, test_iter1)
evaluate_no(model, test_iter2)

100%|██████████| 912/912 [00:05<00:00, 155.72it/s]


tensor(0.9101, device='cuda:0')


100%|██████████| 912/912 [00:05<00:00, 158.58it/s]

tensor(0.0044, device='cuda:0')


tensor(0.0044, device='cuda:0')